In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# open, read, and close text file
with open('/users/salah/documents/mini-gpt/data/tiny-shakespeare.txt', 'r', encoding='utf-8') as file:
    text = file.read()
    
len(text)

1115394

In [3]:
# get unique characters from text
chrs = sorted(list(set(text)))
vocab_size = len(chrs)

# map characters to integers
chr_to_int = {ch: i for i, ch in enumerate(chrs)}
  
# map integers to characters  
int_to_chr = {i: ch for i, ch in enumerate(chrs)}
    
print(chrs)
print(vocab_size)
print(chr_to_int)
print(int_to_chr)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65
{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11

In [4]:
def encode(text: str) -> list[int]:
    """Tokenize characters in text to integers.

    Args:
        text (str): Any string.
    Returns:
        list[int]: Token list of integers.
    """
    return [chr_to_int[ch] for ch in text]
    

def decode(tokens: list[int]) -> str:
    """Revert token integers to text.

    Args:
        tokens (list[int]): List of token integers.

    Returns:
        str: Text derived from tokens.
    """
    return ''.join([int_to_chr[i] for i in tokens])

print(encode('hello world.'))
print(decode(encode('hello world.')))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 8]
hello world.


In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [6]:
# split token data into train & validation sets
n = int(0.9 * len(data))
train = data[:n]
valid = data[n:]

In [7]:
torch.manual_seed(1337)

def get_batch(data: torch.tensor, batch_size: int, block_size: int) -> tuple[torch.tensor]:
    """Returns 

    Args:
        data (torch.tensor): Token tensor.
        batch_size (int): Number of batches of contexts to be processed.
        block_size (int): Length of the context of characters.

    Returns:
        tuple[torch.tensor]: Stacked context and target tensors of dimension block_size x block_size. 
    """
    ix = torch.randint(high=len(data) - block_size, size=(batch_size,))
    j, k = [], []
    for i in ix:
        j.append(data[i:i+block_size])
        k.append(data[i+1:i+block_size+1])
    x = torch.stack(j)
    y = torch.stack(k)
    return x, y

x_train, y_train = get_batch(data=train, batch_size=4, block_size=8)
print(x_train, y_train)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]) tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [8]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    """_summary_

    Superclass: nn.Module
    """
    
    def __init__(self, vocab_size: int) -> None:
        """Initialize a BigramLanguageModel with a token embedding table.

        Args:
            vocab_size (int): _description_
        """
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        """_summary_

        Args:
            idx (_type_): _description_
            targets (_type_): _description_
        """
        # idx & targets are batch x time dimensional tensor
        logits = self.token_embedding_table(idx) # <- batch x time x channel dimensional
        
        if targets == None:
            loss = None
        else:
            b, t, c = logits.shape
            logits = logits.view(b*t, c)
            targets = targets.view(b*t)
            loss = F.cross_entropy(logits, targets) # <- expects batch x time dimensional only
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        """_summary_

        Args:
            idx (_type_): _description_
            max_new_tokens (_type_): _description_
        """
        # idx is b x t array of indices in the current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (b x c)
            # apply Softmax to get probablities
            probs = F.softmax(logits, dim=1) # (b x c)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (b x 1)
            # append sampled idx to running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (b x t+1)
        return idx
            
            

In [9]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(x_train, y_train)
print(logits.shape)
print(loss)

idx = torch.ones((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)
 -KIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [10]:
# create optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [11]:
batch_size = 32
for steps in range(10000):
    
    # sample a batch of data
    x_train, y_train = get_batch(data=train, batch_size=batch_size, block_size=8)
    
    # evaluate the loss
    logits, loss = m(x_train, y_train)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.382369041442871


In [12]:
print(decode(m.generate(idx, max_new_tokens=500)[0].tolist()))

 lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecorro llaus a!
OLeneerithesinthengove fal amas trr
TI ar I t, mes, n IUSt my w, fredeeyove
THek' merer, dd
We ntem lud engitheso; cer ize helorowaginte the?
Thak orblyoruldvicee chot, p,
Bealivolde Th li


## Self-attention

In [13]:
torch.manual_seed(1337)

b, t, c = 4, 8, 2
x = torch.randn(b, t, c)
x.shape

torch.Size([4, 8, 2])

In [14]:
## Weighted aggregation method 1

xbow = torch.zeros((b, t, c)) # bow = bag of words, ie each token has additional features
for i in range(b):
    for j in range(t):
        xprev = x[i, :j+1] # dim (t, c)
        xbow[i, j] = torch.mean(xprev, dim=0)
        
print(x[0], xbow[0])

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]]) tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [15]:
torch.manual_seed(1337)

a = torch.tril(torch.ones(3, 3)) # get lower triangle including diagonal
a = a / torch.sum(a, 1, keepdim=True) # create matrix of average weights
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print(c)


tensor([[5.0000, 7.0000],
        [3.5000, 3.5000],
        [4.0000, 3.3333]])


In [16]:
## Weighted aggregation method 2

torch.manual_seed(1337)

weights = torch.tril(torch.ones(t, t))
weights = weights / weights.sum(1, keepdim=True)
xbow2 = weights @ x # (t, t) @ (b, t, c) (pytorch)-> (b, t, t) @ (b, t, c) -> (b, t, c)
torch.allclose(xbow, xbow2)

True

In [17]:
## Weighted aggregation method 3

torch.manual_seed(1337)

tril = torch.tril(torch.ones(t, t))
weights = torch.zeros(t, t)
weights = weights.masked_fill(tril==0, float('-inf'))
weights = F.softmax(weights, dim=1)
xbow3 = weights @ x
torch.allclose(xbow, xbow3)

True

In [18]:
torch.manual_seed(1337)

b, t, c = 4, 8, 32
x = torch.randn(b, t, c)

head_size = 16
key = nn.Linear(c, head_size, bias=False)
query = nn.Linear(c, head_size, bias=False)
value = nn.Linear(c, head_size, bias=False)
k = key(x) # b x t x 16
q = query(x) # b x t x 16
v = value(x)
weights = q @ k.transpose(-2, -1) # (b x t x 16) @ (b, 16, t) -> (b, t, t)

tril = torch.tril(torch.ones(t, t))
weights = weights.masked_fill(tril==0, float('-inf'))
weights = F.softmax(weights, dim=1)
out = weights @ v

weights[0]

tensor([[0.0248, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0052, 0.0091, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0521, 0.0135, 0.2482, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3171, 0.0214, 0.1642, 0.1188, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0412, 0.0487, 0.1046, 0.0742, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1060, 0.5347, 0.2059, 0.1030, 0.7402, 0.0192, 0.0000, 0.0000],
        [0.4298, 0.3409, 0.1769, 0.2027, 0.0480, 0.8472, 0.2329, 0.0000],
        [0.0238, 0.0316, 0.1002, 0.5013, 0.0117, 0.1336, 0.7671, 1.0000]],
       grad_fn=<SelectBackward0>)